In [1]:
import sqlite3
import json

In [2]:
genesisinfo = dict()

conn = sqlite3.connect('capinfo.sqlite')
c = conn.cursor()
c.execute('SELECT DISTINCT genesiscode, capture, pakket, panel, cnvdetectie, printcnv, mozaiekdetectie FROM capdb')

for out in c.fetchall():
    if out[0] == 'CAPTEST':
        continue
    capture, captureversie = out[1].split('v')    
    pakket, pakketversie = out[2].split('v')
    panel = out[3]
    if panel == 'False':
        panel = None
        panelversie = None
    else:
        panel = panel.replace('typeA', '')
        panel, panelversie = panel.split('v')
        panelversie = int(panelversie)
    cnvdetectie = out[4]
    printcnv = out[5]
    mozaiekdetectie = out[6]
    genesisinfo[out[0]] = (capture, pakket, panel, cnvdetectie, printcnv, mozaiekdetectie)
    
conn.close()
genesisinfo

{'AGP': ('AGPPVM', 'AGP', None, 1, 0, 0),
 'ALS': ('NEURO', 'ALS', 'ALS', 1, 0, 0),
 'ARTM': ('SCD', 'SCD', None, 1, 1, 0),
 'BLALBI': ('BH', 'ALBI', None, 1, 0, 0),
 'BLCRD': ('BH', 'BH', 'CRD', 1, 0, 0),
 'BLLCA': ('BH', 'BH', 'LCA', 1, 0, 0),
 'BLOVR': ('BH', 'BH', 'OVR', 1, 0, 0),
 'BLRP': ('BH', 'BH', 'RP', 1, 0, 0),
 'CM': ('CM', 'CM', None, 1, 1, 0),
 'CWS': ('COWSO', 'COW', None, 0, 0, 0),
 'DL': ('DL', 'DL', None, 1, 1, 0),
 'HT': ('CHC', 'HT', None, 1, 0, 0),
 'LIMGIR': ('NEURO', 'LIMBG', 'OVR', 1, 0, 0),
 'LMP': ('LYMPH', 'LYMPH', None, 0, 0, 0),
 'LQT': ('SCD', 'LQT', None, 1, 1, 0),
 'MTBL': ('MBS', 'MBS', None, 0, 0, 0),
 'NEUROP': ('NEURO', 'CMT', 'CMT', 1, 0, 0),
 'PCH': ('NEURO', 'PCH', 'PCH', 1, 0, 0),
 'POLYPL': ('AGPPVM', 'PP', None, 1, 0, 0),
 'POLYPP': ('AGPPVM', 'PP', None, 1, 0, 0),
 'PRECON': ('PCO', 'PCO', 'BMUT', 1, 1, 0),
 'SO': ('COWSO', 'SO', None, 0, 0, 1),
 'VMF': ('AGPPVM', 'VM', None, 1, 0, 0)}

In [3]:
aandoeningeninfo = dict()

conn = sqlite3.connect('capinfo.sqlite')
c = conn.cursor()
c.execute('SELECT DISTINCT genesiscode, aandoening FROM capdb')

for out in c.fetchall():
    if out[0] == 'CAPTEST':
        continue
    genesis = out[0]
    aandoening = out[1]
    aandoeningeninfo[genesis] = aandoening
    
conn.close()
aandoeningeninfo

{'AGP': 'Arthrogrypose',
 'ALS': 'Amyotrofische Laterale Sclerose',
 'ARTM': 'Aritmie',
 'BLALBI': 'Albinisme',
 'BLCRD': 'ConeRod Dystrophie',
 'BLLCA': 'Leber Congenital Amaurosis',
 'BLOVR': 'Blindheid',
 'BLRP': 'Retinitis Pigmentosa',
 'CM': 'Cardiomyopathie',
 'CWS': 'Cowden syndroom',
 'DL': 'Dyslipidemie',
 'HT': 'Hypothyreodie',
 'LIMGIR': 'Limb-girdle spierdystrofie',
 'LMP': 'Lymphoedeem',
 'LQT': 'Lange QT-syndroom',
 'MTBL': 'Metabole aandoeningen',
 'NEUROP': ' Charcot Marie Tooth',
 'PCH': 'Pontocerebellaire Hypoplasie',
 'POLYPL': 'Polyposis indicatie lynch',
 'POLYPP': 'Polyposis',
 'PRECON': 'Preconceptie screening',
 'SO': 'Segmental Overgrowth',
 'VMF': 'Vaatmalformatie'}

In [4]:
capinfo = dict()

conn = sqlite3.connect('capinfo.sqlite')
c = conn.cursor()
c.execute('SELECT DISTINCT capture, OID, lot, verdund FROM capdb')

for out in c.fetchall():
    if out[0] == 'CAPv1':
        continue
    
    c.execute('SELECT DISTINCT capturesize FROM genes WHERE capture="{}"'.format(out[0]))
    size = c.fetchone()
    c.execute('SELECT genen FROM genes WHERE capture="{}"'.format(out[0]))
    genes = list()
    for _ in c.fetchall():
        for gene in json.loads(_[0]):
            genes.append(gene)
    genes = list(set(genes))
    capinfo[out[0]] = int(out[1].replace('OID', '')), out[2], out[3], size[0], genes

conn.close()
capinfo['COWSOv1']

(43135,
 111514901,
 0,
 76000,
 ['PIK3CA',
  'PIK3R2',
  'SDHD',
  'PTEN',
  'TCS1',
  'AKT1',
  'SDHB',
  'MTOR',
  'AKT3',
  'TCS2'])

In [5]:
pakketinfo = dict()

conn = sqlite3.connect('capinfo.sqlite')
c = conn.cursor()
c.execute('SELECT DISTINCT pakket FROM capdb')

for out in c.fetchall():
    if out[0] == 'CAPv1':
        continue
    c.execute('SELECT DISTINCT pakketsize, genen FROM genes WHERE pakket="{}"'.format(out[0]))
    geneout = c.fetchone()
    size = geneout[0]
    genes = geneout[1]
    pakketinfo[out[0]] = size, genes

conn.close()
pakketinfo['PCOv2']

(145773,
 '["AGA", "ALDH3A2", "ARSA", "ASPA", "ASS1", "BCKDHB", "BLM", "CBS", "CFTR", "CLN3", "CLN5", "COL17A1", "CRTAP", "DHCR7", "ELP1", "FANCC", "G6PC", "GAA", "GALC", "HADHA", "HBB", "HEXA", "HGSNAT", "HSD17B4", "IDUA", "IVD", "LAMA3", "LAMB3", "LAMC2", "MAN2B1", "MCOLN1", "MLC1", "MUSK", "NBN", "NEB", "NPC1", "PEX1", "PEX12", "PEX7", "PMM2", "POLG", "PPT1", "SACS", "SGSH", "SMN1", "SMPD1", "TMEM216", "TPP1", "TSEN54", "TTPA"]')

In [6]:
panelinfo = dict()

conn = sqlite3.connect('capinfo.sqlite')
c = conn.cursor()
c.execute('SELECT DISTINCT panel FROM capdb')

for out in c.fetchall():
    if out[0] == 'False':
        continue
    c.execute('SELECT DISTINCT panelsize, agenen FROM genes WHERE panel="{}"'.format(out[0]))
    geneout = c.fetchone()
    size = geneout[0]
    genes = geneout[1]
    panelinfo[out[0]] = size, genes

conn.close()
panelinfo['LCAtypeAv1']

(52103,
 '["AIPL1", "CABP4", "CEP290", "CRB1", "CRX", "DTHD1", "GDF6", "GUCY2D", "IMPDH1", "IQCB1", "KCNJ13", "LCA5", "LRAT", "NMNAT1", "OTX2", "PRPH2", "RD3", "RDH12", "RPE65", "RPGRIP1", "SPATA7", "TULP1"]')

In [7]:
genesis = """CREATE TABLE genesis (
genesis TEXT NOT NULL,
capture TEXT NOT NULL,
pakket TEXT NOT NULL,
panel TEXT,
cnvscreening BOOLEAN NOT NULL DEFAULT 0,
cnvdiagnostiek BOOLEAN NOT NULL DEFAULT 0,
mozaiekdiagnostiek BOOLEAN NOT NULL DEFAULT 0,
PRIMARY KEY (genesis)
)
"""

In [8]:
aandoeningen = """CREATE TABLE aandoeningen (
genesis TEXT NOT NULL,
aandoening TEXT NOT NULL,
PRIMARY KEY (genesis)
)
"""

In [9]:
captures = """CREATE TABLE captures (
capture TEXT NOT NULL,
versie INTEGER NOT NULL,
oid INTEGER NOT NULL,
lot INTEGER NOT NULL,
verdund BOOLEAN NOT NULL,
grootte INTEGER NOT NULL,
genen TEXT,
PRIMARY KEY (capture, versie, lot)
)
"""

In [10]:
pakketten = """CREATE TABLE pakketten (
pakket TEXT NOT NULL,
versie INTEGER NOT NULL,
grootte INTEGER,
genen TEXT,
PRIMARY KEY (pakket, versie)
)
"""

In [11]:
panels = """CREATE TABLE panels (
panel TEXT NOT NULL,
versie INTEGER NOT NULL,
grootte INTEGER,
genen TEXT,
PRIMARY KEY (panel, versie)
)
"""

In [12]:
def execute_statement(c, sql):
    try:
        c.execute(sql)
    except sqlite3.OperationalError as e:
        print(e)
    except sqlite3.IntegrityError as e:
        print(e)
    else:
        conn.commit()

conn = sqlite3.connect('captures.sqlite')
c = conn.cursor()

execute_statement(c, genesis)
execute_statement(c, aandoeningen)
execute_statement(c, captures)
execute_statement(c, pakketten)
execute_statement(c, panels)
conn.close()

In [13]:
conn = sqlite3.connect('captures.sqlite')
c = conn.cursor()

for k,v in genesisinfo.items():
    genesis = k
    capture = v[0]
    pakket = v[1]
    panel = v[2]
    cnvscreen = v[3]
    cnvdiag = v[4]
    mozaiek = v[5]
    
    sql = "INSERT INTO genesis VALUES ('{}', '{}', '{}', '{}', {}, {}, {})".format(genesis, capture, pakket, panel,
                                                                       cnvscreen, cnvdiag, mozaiek)
    execute_statement(c, sql)
    
conn.close()

In [14]:
conn = sqlite3.connect('captures.sqlite')
c = conn.cursor()

for genesis, aandoening in aandoeningeninfo.items():
    sql = "INSERT INTO aandoeningen VALUES ('{}', '{}')".format(genesis, aandoening)
    execute_statement(c, sql)
    
conn.close()

In [15]:
conn = sqlite3.connect('captures.sqlite')
c = conn.cursor()

for k,v in capinfo.items():
    capture, versie = k.split('v')
    oid = v[0]
    lot = v[1]
    verdund = v[2]
    grootte = v[3]
    genen = v[4]
    sql = "INSERT INTO captures VALUES ('{}', {}, {}, {}, {}, {}, '{}')".format(capture, versie, oid, lot, 
                                                                                verdund, grootte, json.dumps(genen))
    execute_statement(c, sql)
    
conn.close()

In [16]:
conn = sqlite3.connect('captures.sqlite')
c = conn.cursor()

for k,v in pakketinfo.items():
    pakket, versie = k.split('v')
    grootte = v[0]
    genen = v[1]
    sql = "INSERT INTO pakketten VALUES ('{}', {}, {}, '{}')".format(pakket, versie, grootte, genen)
    execute_statement(c, sql)
    
conn.close()

In [17]:
conn = sqlite3.connect('captures.sqlite')
c = conn.cursor()

for k,v in panelinfo.items():
    panel, versie = k.split('v')
    panel = panel.replace('typeA', '')
    grootte = v[0]
    genen = v[1]
    sql = "INSERT INTO panels VALUES ('{}', {}, {}, '{}')".format(panel, versie, grootte, genen)
    execute_statement(c, sql)
    
conn.close()